In [14]:
import argparse
import json
import os
import copy
import re

In [15]:
def predict(data):
    predictions = []
    for d in data:
        pred = {
            'ID': d['ID'],
            'acronyms': [],
            'long-forms': []
        }
        tokens = d['text'].split()
        for i, t in enumerate(tokens):            
#             t2 = copy.deepcopy(t) ## DOES NOT WORK; Thanks to Python referencing!
#             t2 = copy.copy(t) ## DOES NOT WORK; Thanks to Python referencing!
#             t2 = t

            t2 = (t + '.')[:-1] ## Works! 
    
#             t2 = '%s' % t ## DOES NOT WORK; Thanks to Python referencing!
#             t2 = t2.replace(r"\(.*\)","")
#             re.sub(r'\([^()]*\)', '', t2)
            t2 = t2.replace('(',' ').replace(')',' ')
#             t2 = t2.replace('(',' ').replace(')',' ').replace(',',' ').strip()
#             print(t2)
            if len(t2) > 0:
                if t2[0].isupper() and len([c for c in t2 if c.isupper()])/len(t2) > 0.5 and 2 <= len(t2) <= 10:
#                     print(d['ID'], d['text'])
#                     print(id(t2),id(t))
#                     print(i, t2, t)
                    pred['acronyms'].append([sum([len(w)+1 for w in tokens[:i]]),sum([len(w)+1 for w in tokens[:i]])+len(t)])
                    if t.startswith('(') and t.endswith(')'): ## TRY TO REPAIR THIS!
#                         print("found")
                        uppers = []
                        for c in t:
                            if c.isupper():
                                uppers.append(c)
                        candids = tokens[i-len(uppers):i]
    #                     print("candids", i, candids)
                        match = True
                        for j, candid in enumerate(candids):
                            if candid[0].lower() != uppers[j].lower():
                                match = False
                        if match:
                            pred['long-forms'].append([sum([len(w)+1 for w in tokens[:i-len(uppers)]]),sum([len(w)+1 for w in tokens[:i-1]])+len(tokens[i-1])])
        predictions.append(pred)
    return predictions

In [16]:
## 14 files

files = ["../data/english/legal/train.json", "../data/english/legal/dev.json", "../data/english/scientific/train.json", 
         "../data/english/scientific/dev.json", "../data/danish/train.json", "../data/danish/dev.json", "../data/french/train.json",
         "../data/french/dev.json", "../data/persian/train.json", "../data/persian/dev.json", 
         "../data/spanish/train.json", "../data/spanish/dev.json", "../data/vietnamese/train.json", "../data/vietnamese/dev.json"]

for f in files:
    head, tail = os.path.split(f)
    dd = head.replace('..','')
    dd = head.replace('/','')
    dd = dd.replace('data','')
#     print(dd+tail[:-5]+"results.json")
    with open(f) as file:
        data = json.load(file)
    predictions = predict(data) ## Use the function above to predict based on the "Word capitalized over 60% and above is an acronym rule"
    with open("../results/baseline/"+dd+tail[:-5]+".json", 'w') as file:
        json.dump(predictions, file) ## Dump the prediction in a file

In [17]:
results = ["../results/baseline/englishlegaltrain.json", "../results/baseline/englishlegaldev.json", "../results/baseline/englishscientifictrain.json",
           "../results/baseline/englishscientificdev.json", "../results/baseline/danishtrain.json", "../results/baseline/danishdev.json", "../results/baseline/frenchtrain.json", "../results/baseline/frenchdev.json", "../results/baseline/persiantrain.json", "../results/baseline/persiandev.json",
           "../results/baseline/spanishtrain.json", "../results/baseline/spanishdev.json", "../results/baseline/vietnamesetrain.json", "../results/baseline/vietnamesedev.json"]

In [18]:
def run_evaluation(file1g, file2p, verbose=False):
    with open(file1g) as file:
        gold = dict([(d['ID'], {'acronyms':d['acronyms'],'long-forms':d['long-forms']}) for d in json.load(file)])
    with open(file2p) as file:
        pred = dict([(d['ID'], {'acronyms':d['acronyms'],'long-forms':d['long-forms']}) for d in json.load(file)])
        print(file2p)
        pred = [pred[k] for k,v in gold.items()]
        gold = [gold[k] for k,v in gold.items()]
    p, r, f1 = score_phrase_level(gold, pred, verbos=verbose)
    return p, r, f1

def score_phrase_level(key, predictions, verbos=False):
    gold_shorts = set()
    gold_longs = set()
    pred_shorts = set()
    pred_longs = set()

    def find_phrase(seq, shorts, longs):
        for i, data in enumerate(seq):
            for sh in data['acronyms']:
                shorts.add(str(i)+'#'+str(sh[0])+'-'+str(sh[1]))
            for lf in data['long-forms']:
                longs.add(str(i)+'#'+str(lf[0])+'-'+str(lf[1]))

    find_phrase(key, gold_shorts, gold_longs)
    find_phrase(predictions, pred_shorts, pred_longs)

    def find_prec_recall_f1(pred, gold):
        correct = 0
        for phrase in pred:
            if phrase in gold:
                correct += 1
        # print(correct)
        prec = correct / len(pred) if len(pred) > 0 else 1
        recall = correct / len(gold) if len(gold) > 0 else 1
        f1 = 2 * prec * recall / (prec + recall) if prec+recall > 0 else 0
        return prec, recall, f1

    prec_short, recall_short, f1_short = find_prec_recall_f1(pred_shorts, gold_shorts)
    prec_long, recall_long, f1_long = find_prec_recall_f1(pred_longs, gold_longs)
    precision_micro, recall_micro, f1_micro = find_prec_recall_f1(pred_shorts.union(pred_longs), gold_shorts.union(gold_longs))

    precision_macro = (prec_short + prec_long) / 2
    recall_macro = (recall_short + recall_long) / 2
    f1_macro = 2*precision_macro*recall_macro/(precision_macro+recall_macro) if precision_macro+recall_macro > 0 else 0

    if verbos:
        print('Shorts: P: {:.2%}, R: {:.2%}, F1: {:.2%}'.format(prec_short, recall_short, f1_short))
        print('Longs: P: {:.2%}, R: {:.2%}, F1: {:.2%}'.format(prec_long, recall_long, f1_long))
        print('micro scores: P: {:.2%}, R: {:.2%}, F1: {:.2%}'.format(precision_micro, recall_micro, f1_micro))
        print('macro scores: P: {:.2%}, R: {:.2%}, F1: {:.2%}'.format(precision_macro, recall_macro, f1_macro))

    return precision_macro, recall_macro, f1_macro

In [19]:
!rm ../results/baseline/allresults.tsv ## Delete old file
with open("../results/baseline/allresults.tsv", 'a') as outfile:
    outfile.write("SET\tPrecision\tRecall\tF1-Score\n")
for gold,pred in zip(files,results):
    p, r, f1 = run_evaluation(gold, pred)
    new = "{:.2}".format(p)
    head, tail = os.path.split(pred)
    with open("../results/baseline/allresults.tsv", 'a') as outfile:
        outfile.write(tail[:-5]+"\t{:.2%}\t{:.2%}\t{:.2%}\n".format(p,r,f1))

../results/baseline/englishlegaltrain.json
../results/baseline/englishlegaldev.json
../results/baseline/englishscientifictrain.json
../results/baseline/englishscientificdev.json
../results/baseline/danishtrain.json
../results/baseline/danishdev.json
../results/baseline/frenchtrain.json
../results/baseline/frenchdev.json
../results/baseline/persiantrain.json
../results/baseline/persiandev.json
../results/baseline/spanishtrain.json
../results/baseline/spanishdev.json
../results/baseline/vietnamesetrain.json
../results/baseline/vietnamesedev.json
